In [1]:
#https://blog.dataiku.com/2016/10/08/machine-learning-markov-chains-generate-clinton-trump-quotes
#TODO: get a lot more text for each author, 
#implement starting a phrase with a <sentence begin> and ending it with a <sentence end>
#implement Backoff, 
#implement word weights (right now the ngram densities are too low for weights to do much) for overall and in-dialogue
#graph search
#A state is the N words at the end of the sentence, and the list of magic words. A transition between states is the adding of a new word to the end of res. 
#Ngrams alg defines neighbors
import nltk
import random

In [23]:
N = 3
MAX_DIALOGUE_LEN = 35
AUTHOR_AUG = "Augustine"
AUTHOR_ZZ = "Zhuangzi"
MAGIC_WORDS = set([])#["god","man","world","death","heaven","life","time","body","mind"])
grams1 = {}
grams2 = {}
grams3 = {}
empty_words = ['.',',',':','i','the','should','it','did','for','by','this','they','them','because','so','is','and','a','with','to','\'', '\"','be','not','no','thus','in','have','as','but','from','on','do','at','or','an','will','my','so','if', 'these', 'those','of','was','there','then','what','we','its','it\'s', 'as','into','that','may','are','he','can','how','such','were','which','\'','who','him','any','one','his','like','about','when','has','things','yet','all','said']

In [3]:
class gramlib(object):
    def __init__(self, gdict = [], start_list = []):
        self.gdict = gdict
        self.startList = start_list
    def getPost(self,pre):
        return self.gdict[pre]
    def getStarter(self):
        return random.choice(self.startList)

In [4]:
def initialize_grams(grams, textfile): #initialize the dicts that store the markov probabilities for N-word chains
    f = open(textfile)
    s = f.read().replace('(','').replace(')','')
    t = nltk.word_tokenize(s)
    start_list = []
    for i in range(len(t)-(N)):
        trigram = tuple(t[x].lower() for x in range(i,i+N))
        if(t[i-1]=="." or t[i-1]=="!" or t[i-1]=="?"):
            if(t[i]!=" " and t[i]!="," and t[i]!="\'" and t[i]!="."):
                start_list.append(trigram)
        if trigram in grams.keys():
            grams[trigram].add(t[i+N].lower())
        else:
            grams[trigram] = set([t[i+N].lower()])
    g = gramlib(grams, start_list)
    return g

            
def ngram_generate(grams): #generate text from a dict passed in
    nwords = 0
    start = grams.getStarter()
    res = list(start)
    while(nwords < 100):
        pre = tuple(res[(-1*N):])
        nextword = random.choice(list(grams[pre]))
        res.append(nextword)
        if nextword == '.' or nextword =='?': break
        nwords +=1
    if nwords ==100: res.append('-')
    return " ".join(res).replace(' .', '.').replace(' ,', ',').replace(' ;',';').replace(' ?', '?').replace(' \'', '\'').replace(' !', '!')

def solve_density(grams): #On average, how many words are in the dict for each ngram?
    total = 0.0
    div = 0
    for word in grams.keys():
        if(len(grams[word])>1):
            total += len(grams[word])
            div +=1
    return total/div

In [5]:
#initialize_grams(grams1, 'augustine_full.txt')
#initialize_grams(grams1, 'zhuangzi.txt')
#solve_density(grams1)

In [28]:
class NGramSearchProblem(object):
    def __init__(self, N, grams, magic_words):
        self.N = N
        self.grams = grams
        self.magic_words = magic_words
        self.startActions = None

    # Trivially return 100 if word is magic word, 1 otherwise
    def ngram_cost(self,state):
        if state[-1] in self.magic_words:
            return 30#try different weights
        if state[-1] == "." or state[-1] == "?" or state[-1] == "!":
            return 5
        return 1
    
    def ngram_generate_next_words(self, curr_gram):
        pre = tuple(curr_gram[(-1*N):])
        l =self.grams.getPost(pre)
        pres = []
        for i in range(len(curr_gram)-N):
            pres.append(curr_gram[i:i+N])
        suffix = curr_gram[-1*(N-1):]
        result = [w for w in l if suffix+[w] not in pres]
        final_result = [p for p in result if curr_gram.count(p)<2]
        return final_result
       # print("pres >>>>", pres)
        #print("res >>>>>", result)
        #if (len(result)!=len(l)):
         #   print('success')
        return result
    
    def startState(self):
        start = self.grams.getStarter()
        self.startActions = list(start)
        return list(start)

    def isEnd(self, state):
        return len(state) >= MAX_DIALOGUE_LEN or state[-1][-1] == "." or state[-1][-1] == "?" or state[-1][-1] == "!"#max blurb length or last word ends in period
    
    def succAndCost(self, state):
        result = []
        possible_next_words = self.ngram_generate_next_words(state)
        for next_word in possible_next_words:
            next_state = state[:]+[next_word]
            result.append((next_word, next_state, self.ngram_cost(next_state)))
        return result
    def getMagic(self):
        return self.magic_words

In [29]:
def backtrackingSearch(problem):
    bestTotalCost = [float('-inf')]
    bestHistory = [None]
    def recurse(state,curr_history,curr_cost):
        if problem.isEnd(state):
            if curr_cost > bestTotalCost[0]:
                bestTotalCost[0], bestHistory[0] = curr_cost, curr_history 
            return
        for action, next_state, cost in problem.succAndCost(state):
            recurse(next_state, curr_history+[action], curr_cost+cost)
    
    recurse(problem.startState(),problem.startState(),0)
    bestHistory = procStr(" ".join(bestHistory[0]))
   # print("<-----------Best History--------->")
    #print(bestHistory)return 
    #print("<--------------Score------------->")
    #print(bestTotalCost[0])
    return bestHistory

In [30]:
def kBeamsSearch(problem,k = 2500):
    C = []
    for i in range(k):
        s = problem.startState()
        i = 0
        for w in problem.getMagic():
            if w in s:
                i += 30
        C.append((s,s,i))
    complete_beams = []
    while len(complete_beams) < k:
        for j in range(len(C)):
            curr_beam = C[j][:]
            for action, next_state, cost in problem.succAndCost(curr_beam[0]):    
                C.append((next_state, curr_beam[1]+[action],curr_beam[2]+cost))
        C = sorted(C, key=lambda x: x[2])[::-1]
        C = C if len(C) < k else C[:k]
        for beam in C:
            if problem.isEnd(beam[0]):
                complete_beams.append(beam)  
        new_beams = [b for b in C if b not in complete_beams]
        C = new_beams
    res = complete_beams[-1]
    return procStr(" ".join(res[1]))

In [9]:
#blurb = backtrackingSearch(NGramSearchProblem(N,grams1,MAGIC_WORDS)) 

In [16]:
sigs = []
text1 = 'zhuangzi.txt'
g1 = initialize_grams(grams1, text1)
speech1 = kBeamsSearch(NGramSearchProblem(N,g1,set(list(MAGIC_WORDS)+sigs)))
speech1

NameError: name 'procStr' is not defined

In [18]:
#todo - get rid of bad punctuation
#capitalize first letter
#end with a hyphen if they are cut off.
def convo(text1, text2, author1, author2):
    grams1 = {}
    grams2 = {}
    g1 = initialize_grams(grams1, text1)
    g2 = initialize_grams(grams2, text2)
    print('first ngram density: '+ str(solve_density(grams1))+ '\n')
    print('second ngram density: '+ str(solve_density(grams2))+ '\n')
    sigs = []
    for i in range(5):
        print(author1 + ':')
        #speech1 = ngram_generate(grams1)
        #speech1 = backtrackingSearch(NGramSearchProblem(N,g1,set(list(MAGIC_WORDS)+sigs)))
        speech1 = kBeamsSearch(NGramSearchProblem(N,g1,set(list(MAGIC_WORDS)+sigs)))
        sigs = getSigWords(speech1)
        print(speech1)
        print('\n')
        #print(sigs)
        print('\n')
        #speech2 = ngram_generate(grams2)
        #speech2 = backtrackingSearch(NGramSearchProblem(N,g2,set(list(MAGIC_WORDS)+sigs)))
        speech2 = kBeamsSearch(NGramSearchProblem(N,g2,set(list(MAGIC_WORDS)+sigs)))
        sigs = getSigWords(speech2)
        print(author2 + ':')
        print(speech2)
        print('\n')
        #print(sigs)
        print('\n')
        i+=1

In [19]:
convo('zhuangzi.txt','augustine_full.txt', 'Zhuangzi','Augustine')

first ngram density: 2.954440026507621

second ngram density: 3.4217966453397892

Zhuangzi:
Zi-chan said to him, do not know who it is that you will not be angry with it.






KeyboardInterrupt: 

In [31]:
def getSigWords(str):
    s = set(nltk.word_tokenize(str))
    for w in empty_words:
        if w in s: s.remove(w)
    return list(s)

In [32]:
def procStr(str):
    newstr = str.replace('0','').replace('1','').replace('2','').replace('3','').replace('4','').replace('5','').replace('6','').replace('7','').replace('8','').replace('9','').replace('`','').replace('\'','').replace(' .', '.').replace(' ,', ',').replace(' ;',';').replace(' ?', '?').replace(' \'', '\'').replace(' !', '!').replace(' i ', ' I ').replace(' i ', ' I ').replace(' ;',';')
    if (newstr[-1]!='.' and newstr[-1]!='?' and newstr[-1]!='!'):
        newstr = newstr[:] + '-'
    return newstr[0].upper() + newstr[1:]

In [33]:
def triple_conv(text1,text2,text3,author1,author2,author3,length):
    grams1 = {}
    grams2 = {}
    grams3 = {}
    g1 = initialize_grams(grams1, text1)
    g2 = initialize_grams(grams2, text2)
    g3 = initialize_grams(grams3, text3)
    print('first ngram density_v2: '+ str(solve_density(grams1))+ '\n')
    print('second ngram density_v2: '+ str(solve_density(grams2))+ '\n')
    print('third ngram density_v2: '+ str(solve_density(grams3))+ '\n')
    sigs = []
    for i in range(length):
        print(author1 + ':')
        #speech1 = ngram_generate(grams1)
        speech1 = kBeamsSearch(NGramSearchProblem(N,g1,set(list(MAGIC_WORDS)+sigs)))
        sigs = getSigWords(speech1)
        print(speech1)
        print('\n')
        #print(sigs)
        print('\n')
        #speech2 = ngram_generate(grams2)
        speech2 = kBeamsSearch(NGramSearchProblem(N,g2,set(list(MAGIC_WORDS)+sigs)))
        sigs = getSigWords(speech2)
        print(author2 + ':')
        print(speech2)
        print('\n')
        #print(sigs)
        print('\n')
        speech3 = kBeamsSearch(NGramSearchProblem(N,g3,set(list(MAGIC_WORDS)+sigs)))
        sigs = getSigWords(speech3)
        print(author3 + ':')
        print(speech3)
        print('\n')
        #print(sigs)
        print('\n')
        i+=1
    

In [34]:
triple_conv('zhuangzi.txt','augustine_full.txt','plato_full.txt','Zhuangzi', 'Augustine', 'Plato',3)

first ngram density_v2: 2.954440026507621

second ngram density_v2: 3.4217966453397892

third ngram density_v2: 3.7854438458709243

Zhuangzi:
The one of these conditions and the other again, and he said, what do you mean?




Augustine:
Other things of this life again expose himself to them, the other, so that there are both good and bad.




Plato:
Very good. or again, when a good and bad?




Zhuangzi:
When good order prevails in the world, and the other again, and he does not know how.




Augustine:
Again, if he does not know that they are good, and the other with the other two.




Plato:
And does not know the other, if you know of any other art?




Zhuangzi:
He does not know how it is that you do not know where they are without the dao.




Augustine:
 here you see he does not know that it is not, as you say, and to know.




Plato:
You will see, if you say that he does not know letters?






In [14]:
def player_conv(text, name):
    pname = input("Enter your name:\n")
    grams1 = {}
    g1 = initialize_grams(grams1, text)
    sigs = []
    while(true):
        print("\n" + pname + ": ")
        speech = input()
        if(speech == "end"):
            break
        #print(speech)
        sigs = getSigWords(speech)
        speech = kBeamsSearch(NGramSearchProblem(N,g1,set(list(MAGIC_WORDS)+sigs)))
        print(" \n" + name + ": "+ speech + "\n")
        

In [6]:
player_conv('plato_full.txt', 'Plato')

Enter your name:
Sam


NameError: name 'initialize_grams' is not defined

In [35]:
z = ['s','a','b','t','n','z','t']

In [36]:
z.remove([a for a in g])

ValueError: list.remove(x): x not in list

In [49]:
a = []

In [50]:
a.append('a')

In [51]:
b = ['b']

In [52]:
a+b

['a', 'b']